In [1]:
# Import used labraries
import pandas as pd # pandas 
import numpy as np # numpy 
from sklearn.model_selection import KFold # kfold for model validation
from sklearn.model_selection import train_test_split # for data spliting
from sklearn.preprocessing import StandardScaler # for data normalization 
import matplotlib.pyplot as plt


# Sklearn modules 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Sklearn accuracy 
from sklearn.metrics import accuracy_score
from collections import Counter


#set display 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df_data = pd.read_csv("../dat/2021VAERSDATA.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
df_sympt = pd.read_csv("../dat/2021VAERSSYMPTOMS.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

# use this df to select covid19 vaccines
df_vax = pd.read_csv("../dat/2021VAERSVAX.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [3]:
print('2021VAERSDATA shape ',df_data.shape[0],' rows ' ,df_data.shape[1],' columns ') # DONE
print('2021VAERSSYMPTOMS shape ',df_sympt.shape[0],' rows ' ,df_sympt.shape[1],' columns ') # DONE
print('2021VAERSVAX shape ',df_vax.shape[0],' rows ' ,df_vax.shape[1],' columns ') # DONE

2021VAERSDATA shape  747180  rows  35  columns 
2021VAERSSYMPTOMS shape  999509  rows  11  columns 
2021VAERSVAX shape  791183  rows  8  columns 


In [4]:
# select covid19 vaccines 'PFIZER\\BIONTECH' 'MODERNA' 'JANSSEN' AstraZeneca is not listed here !
vax_names = ['PFIZER\\BIONTECH','MODERNA','JANSSEN']
df_vax_cols = ['VAERS_ID','VAX_MANU']
df_tep = df_vax[(df_vax['VAX_TYPE'] == 'COVID19') & (df_vax['VAX_DOSE_SERIES'] == '3') & (df_vax['VAX_MANU'].isin(vax_names))]
df_vax_selected = df_tep[df_vax_cols]
df_vax_selected.VAX_MANU = pd.Categorical(df_vax_selected['VAX_MANU'])


df_vax_selected.reset_index(drop=True, inplace=True)


C:\Users\eliaa\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
df_vax_selected.head(100)

df_vax_selected.dtypes



VAERS_ID      object
VAX_MANU    category
dtype: object

In [6]:
import random
l1 = list(pd.unique( df_sympt.SYMPTOM1.values.ravel()))
l2 = list(pd.unique( df_sympt.SYMPTOM2.values.ravel()))
l3 = list(pd.unique( df_sympt.SYMPTOM3.values.ravel()))
l4 = list(pd.unique( df_sympt.SYMPTOM4.values.ravel()))
l5 = list(pd.unique( df_sympt.SYMPTOM5.values.ravel()))

n = 30
# print('SYMPTOM1: ' , random.sample(l1, n), '\n \n') 
# print('SYMPTOM2: ' , random.sample(l2, n), '\n \n')
# print('SYMPTOM3: ' , random.sample(l3, n), '\n \n')
# print('SYMPTOM4: ' , random.sample(l4, n), '\n \n')
# print('SYMPTOM5: ' , random.sample(l5, n), '\n \n')
# print('SYMPTOM6: ' , random.sample(l6, n), '\n \n')
# print('SYMPTOM7: ' , random.sample(l7, n), '\n \n')
best_7sy = []
best_7sy.append(df_sympt.SYMPTOM1.value_counts()[:n].sort_values(ascending=False).index.tolist())
best_7sy.append(df_sympt.SYMPTOM2.value_counts()[:n].sort_values(ascending=False).index.tolist())
best_7sy.append(df_sympt.SYMPTOM3.value_counts()[:n].sort_values(ascending=False).index.tolist())
best_7sy.append(df_sympt.SYMPTOM4.value_counts()[:n].sort_values(ascending=False).index.tolist())
best_7sy.append(df_sympt.SYMPTOM5.value_counts()[:n].sort_values(ascending=False).index.tolist())

def flatten(l):
    res = []
    for x in l:
        for j in x:
            res.append(j)
    return res

print('most common in symptum 1 is:\n',df_sympt.SYMPTOM1.value_counts()[:n].sort_values(ascending=False).index.tolist(),'\n' +'--'*50)
print('most common in symptum 2 is:\n',df_sympt.SYMPTOM2.value_counts()[:n].sort_values(ascending=False).index.tolist(),'\n' +'--'*50)
print('most common in symptum 3 is:\n',df_sympt.SYMPTOM3.value_counts()[:n].sort_values(ascending=False).index.tolist(),'\n' +'--'*50)
print('most common in symptum 4 is:\n',df_sympt.SYMPTOM4.value_counts()[:n].sort_values(ascending=False).index.tolist(),'\n' +'--'*50)
print('most common in symptum 5 is:\n',df_sympt.SYMPTOM5.value_counts()[:n].sort_values(ascending=False).index.tolist(),'\n' +'--'*50)



# df_sympt_filter = (df_sympt[(df_sympt['SYMPTOM1'].isin(selcted_symptoms)) | 
#                     (df_sympt['SYMPTOM2'].isin(selcted_symptoms)) |
#                     (df_sympt['SYMPTOM3'].isin(selcted_symptoms)) |
#                     (df_sympt['SYMPTOM4'].isin(selcted_symptoms)) |
#                     (df_sympt['SYMPTOM5'].isin(selcted_symptoms))])


# df_sympt_selected = df_sympt[selcted_symptoms]
# df_sympt_selected.VAX_MANU = pd.get_dummies(df_vax_selected['VAX_MANU'])


# df_vax_selected.reset_index(drop=True, inplace=True)
# df_vax_selected.head(10)

# for x , count in Counter(flatten(best_7sy)).most_common(n+3):
#     print(x , ' ' , count)

most common in symptum 1 is:
 ['Chills', 'Arthralgia', 'Dizziness', 'Fatigue', 'Headache', 'COVID-19', 'Asthenia', 'Pyrexia', 'Injection site erythema', 'Pain in extremity', 'Pain', 'Expired product administered', 'Erythema', 'Nausea', 'Rash', 'Product storage error', 'Pruritus', 'Dyspnoea', 'Injection site pain', 'No adverse event', 'Chest pain', 'Chest discomfort', 'Blood test', 'Myalgia', 'Product administered to patient of inappropriate age', 'Hypoaesthesia', 'Back pain', 'Diarrhoea', 'Inappropriate schedule of product administration', 'Body temperature'] 
----------------------------------------------------------------------------------------------------
most common in symptum 2 is:
 ['Headache', 'Fatigue', 'Chills', 'Dizziness', 'Pyrexia', 'Pain', 'Pain in extremity', 'Nausea', 'Injection site pain', 'Dyspnoea', 'SARS-CoV-2 test positive', 'Rash', 'Pruritus', 'Myalgia', 'Diarrhoea', 'Injection site pruritus', 'No adverse event', 'Chest pain', 'Cough', 'Feeling abnormal', 'Astheni

In [ ]:
selcted_symptoms =  ['Chills', 'Pyrexia', 'Dizziness', 'Fatigue', 'Headache', 'Pain in extremity', 'Nausea']

def selector(row):
    selcted_symptoms =  ['Chills', 'Pyrexia', 'Dizziness', 'Fatigue', 'Headache', 'Pain in extremity', 'Nausea']
    if row in selcted_symptoms:
        return row
    else:
        return 'Other'

symptoms_selected_cols = ['VAERS_ID','SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5']
symptoms_cols = ['SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5']
df_sympt_filter = df_sympt[symptoms_selected_cols]
for x in symptoms_cols:
    df_sympt_filter[x] = df_sympt_filter.apply( lambda row : selector(row), axis = 1)

In [ ]:

#df_sympt_filter_selected = df_sympt_filter[symptoms_selected_cols]
df_sympt_filter.head()

In [ ]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]
selcted_symptoms.append('VAERS_ID')
df_temp = filter_rows_by_values(df_sympt,symptoms_cols,selcted_symptoms)

In [ ]:
df_sympt.head(30)

In [ ]:
# df_sympt_filter_selected.dtypes
# df_sympt_filter_before = df_sympt_filter_selected



In [ ]:
# for x in symptoms_cols:
#     df_sympt_filter_selected[x] = df_sympt_filter_selected[x].astype(str)
#     df_sympt_filter_selected[x] = pd.Categorical(df_sympt_filter_selected[x])
#     #df_sympt_filter[x] = pd.factorize(df_sympt_filter[x])[0]
    
# df_sympt_filter_selected[symptoms_cols] = df_sympt_filter_selected[symptoms_cols].apply(lambda x: x.cat.codes, axis = 1)


In [ ]:
def plot_frec_col(df, colName):
    # plot the histogram of frequencies of values in column label

    # convert the column to a list 
    arr_label = df[colName].to_list()

    # count the values of the prev. list 
    label_labels, label_counts = np.unique(arr_label, return_counts=True)

    # set the general font size to 16
    plt.rcParams['font.size'] = '16'

    # set the figure size 
    plt.figure(figsize=(12,8))

    # plot the values as bar histogram 
    plt.bar(label_labels, label_counts, align='center')

    # set the title of the plot and axes names
    tit = 'Histogram of column '+ str(colName)
    plt.title(tit,fontsize=25)
    plt.xlabel(colName)
    plt.ylabel('frequency')
    plt.setp(plt.gca().get_xticklabels(), rotation=70, horizontalalignment='right')
    plt.gca().set_xticks(label_labels)

    # show the plot 
    plt.show()

In [ ]:
plot_frec_col (df_sympt, 'SYMPTOM1')

In [ ]:
df_syptm_cols_corr = df_sympt_filter[symptoms_cols]

df_syptm_cols_corr.head(10)

In [7]:
# After cleaning select top 10 + Unknown
df_data_clean = df_data
df_data['HISTORY'].value_counts()[:20].sort_values(ascending=False)
replace_dist_history= {
    'None':'Unknown',
    'none': 'Unknown',
    'Comments: Unknown':'Unknown',
    'No':'Unknown',
    'asthma':'Asthma',
    'mild asthma':'Asthma',
    'Asthma.':'Asthma',
    'Medical History/Concurrent Conditions: Asthma':'Asthma',
    'no':'Unknown',
    'NONE':'Unknown',
    'none.':'Unknown',
    'unknown':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event':'Unknown',
    'Comments: No medical history was provided by the reporter.':'Unknown',
    'hypertension': 'Hypertension',
    'Medical History/Concurrent Conditions: Hypertension': 'Hypertension',
    'None.' : 'Unknown',
    'diabetes': 'Diabetes',
    'Type 1 diabetes': 'Diabetes',
    'Diabetes type 2': 'Diabetes',
    'Diabetes Type 2': 'Diabetes',
    'Type 1 Diabetes': 'Diabetes',
    'Type 2 Diabetes':'Diabetes',
    'Type 2 diabetes':'Diabetes',
    'Medical History/Concurrent Conditions: Diabetes': 'Diabetes',
    'Medical History/Concurrent Conditions: No adverse event (No reported medical history)':'Unknown',
    'Medical History/Concurrent Conditions: Anxiety':'Anxiety',
    'High blood pressure':'Hypertension',
    'HIGH BLOOD PRESSURE':'Hypertension',
    'HYPERTENSION':'Hypertension',
    'high blood pressure':'Hypertension',
    'High Blood Pressure' : 'Hypertension',
    'High blood pressure.': 'Hypertension',
    'High BP':'Hypertension',
    'Medical History/Concurrent Conditions: Blood pressure high':'Hypertension',
    'hypothyroidism': 'Hypothyroidism',
    'Medical History/Concurrent Conditions: COVID-19':'COVID-19',
    'Medical History/Concurrent Conditions: COVID-19 (Prior to vaccination, was the patient diagnosed with COVID-19?:Yes)':'COVID-19',
    'Medical History/Concurrent Conditions: COVID-19 (Prior to vaccination, was the patient diagnosed with COVID-19?: Yes)':'COVID-19',
    'Medical History/Concurrent Conditions: No adverse event (No medical history reported)':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (No reported medical history.)' : 'Unknown',
    'Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: none':'Unknown',
    'Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: None, Comment: Patient History: No':'Unknown',
    'Medical History/Concurrent Conditions: Rheumatoid arthritis':'Rheumatoid Arthritis',
    'Comments: No medical history was reported by the reporter.':'Unknown',
    'Comments: No medical history information was reported.':'Unknown',
    'Medical History/Concurrent Conditions: Migraine':'Migraines',
    'Mild asthma':'Asthma',
    'NONE REPORTED':'Unknown',
    'none listed':'Unknown',
    'denies':'Denies',
    'Comments: The patient had no known allergies.':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (no reported medical history)':'Unknown',
    'Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: None':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (No recorded medical history)':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (no medical history provided)':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (no medical history reported)':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (No medical history provided)':'Unknown',
    'Comments: No Medical History Information was reported.':'Unknown',
    'Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: None.':'Unknown',
    'Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: No':'Unknown',
    'Medical History/Concurrent Conditions: No adverse reaction':'Unknown',
    'Comments: No medical history was reported':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (No medical history was reported.':'Unknown',
    'Not known':'Unknown',
    'NOT KNOWN':'Unknown',
    'None Reported':'Unknown',
    'None Known':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event.':'Unknown',
    'rheumatoid arthritis':'Rheumatoid Arthritis',
    'Medical History/Concurrent Conditions: Depression':'Depression',
    'Comments: No Medical History was provided by the reporter.':'Unknown',
    'diabetic':'Diabetic',
    'Comments: Medical history was not provided.':'Unknown',
    'Comments: No medical history was provided by reporter.':'Unknown',
    'Medical History/Concurrent Conditions: Arthritis':'Arthritis',
    'Comments: No medical history was provided by the reporter':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (No medical history reported.)':'Unknown',
    'Comments: No medical history reported.':'Unknown',
    'Comments: No Medical History information was reported.' : 'Unknown',
    'Comments: No medical history was reported.':'Unknown',
    'Medical History/Concurrent Conditions: Penicillin allergy' : 'Allergy',
    'Medical History/Concurrent Conditions: Hypothyroidism' : 'Hypothyroidism',
    'UNKNOWN':'Unknown',
    'N/a':'Unknown',
    'None reported':'Unknown',
    'none known':'Unknown',
    'NO':'Unknown',
    'none reported':'Unknown',
    'na':'Unknown',
    'Na':'Unknown',
    'None known':'Unknown',
    'Comments: None':'Unknown',
    'migraines':'Migraines',
    'Rheumatoid arthritis' : 'Rheumatoid Arthritis',
    'arthritis':'Arthritis',
    'obesity':'Obesity',
    'Medical History/Concurrent Conditions: Sulfonamide allergy':'Allergy',
    'Medical History/Concurrent Conditions: Allergy':'Allergy',
    'anxiety':'Anxiety',
    'hypothyroid':'Hypothyroid',
    'high cholesterol':'High cholesterol',
    'Migraine':'Migraines',
    'migraine':'Migraines',
    'ASTHMA':'Asthma',
    'See above':'Unknown',
    'No.':'Unknown',
    'see above':'Unknown',
    'not known':'Unknown',
    'none.':'Unknown',
    '0':'Unknown',
    'depression':'Depression',
    'seasonal allergies':'Allergy',
    'Seasonal allergies':'Allergy',
    'Seasonal Allergies':'Allergy',
    'Penicillin allergy':'Allergy',
    'Medical History/Concurrent Conditions: Seasonal allergy':'Allergy',
    'Medical History/Concurrent Conditions: Drug allergy':'Allergy',
    'Medical History/Concurrent Conditions: Shellfish allergy':'Allergy',
    'Medical History/Concurrent Conditions: Penicillin allergy (Known allergies: Penicillin)':'Allergy',
    'Medical History/Concurrent Conditions: Food allergy':'Allergy',
    'allergies':'Allergy',
    'Medical History/Concurrent Conditions: Shingles': 'Shingles',
    'Medical History/Concurrent Conditions: Cancer':'Cancer',
    'Medical History/Concurrent Conditions: Breast cancer':'Cancer',
    'None listed':'Unknown',
    'Unknown.':'Unknown',
    'GERD':'Gerd',
    'eczema':'Eczema',
    'unk':'Unknown',
    'none noted':'Unknown',
    'same as above':'Unknown',
    'Yes':'Unknown',
    'overweight':'Overweight',
    'UNK':'Unknown',
    'None noted':'Unknown',
    'Comments: No medical history was provided.':'Unknown',
    'Copd':'COPD',
    'copd':'COPD',
    'Medical History/Concurrent Conditions: COPD':'COPD',
    'epilepsy':'Epilepsy',
    'htn':'HTN',
    'Same as above':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (No medical history was reported.)':'Unknown',
    'Non':'Unknown',
    'Nothing':'Unknown',
    'NONE KNOWN':'Unknown',
    'Medical History/Concurrent Conditions: No adverse event (no historical condition was reported)':'Unknown'
}
df_data_clean['HISTORY'] = df_data_clean['HISTORY'].replace(replace_dist_history)

#df_data_clean['HISTORY'].value_counts()[:20].sort_values(ascending=False)


In [8]:
# merging the dataframe to create a dataframe to use in fitting our pre-selected models
replace_dist_OTHER_MEDS = {
  'none':'None',
  'unknown':'None',
  'Unknown':'None',
  'No':'None',
  'NONE':'None',
  'no':'None',
  'UNKNOWN':'None',
  'None.':'None',
  'N/a':'None',
  'None reported':'None',
  'na':'None',
  'Not known':'None',
  'unkown':'None',
  'Na':'None',
  'Yes':'None',
  'NA':'None',
  'synthroid':'Synthroid',
  'None listed':'None',
  'NO':'None',
  'ibuprofen':'Ibuprofen',
  'none reported':'None',
  'none known':'None',
  'None known':'None',
  'NONE REPORTED':'None',
  'not known':'None',
  'Nothing':'None',
  'NOT KNOWN':'None',
  'nothing':'None',
  'None noted':'None',
  'None Known':'None',
  'tylenol':'Tylenol',
  '0':'None',
  'Unk':'None',
  'ZYRTEC [CETIRIZINE HYDROCHLORIDE]':'ZYRTEC'.capitalize(),
  'ATORVASTATIN':'ATORVASTATIN'.capitalize(),
  'XARELTO':'XARELTO'.capitalize(),
  'WELLBUTRIN':'WELLBUTRIN'.capitalize(),
  'ARMOUR THYROID':'ARMOUR THYROID'.capitalize(),
  'METHOTREXATE':'METHOTREXATE'.capitalize(),
  'ADDERALL':'ADDERALL'.capitalize(),
  'LEXAPRO':'LEXAPRO'.capitalize(),
  'LIPITOR':'LIPITOR'.capitalize(),
  'GABAPENTIN':'GABAPENTIN'.capitalize(),
  'METOPROLOL':'METOPROLOL'.capitalize(),
  'SIMVASTATIN':'SIMVASTATIN'.capitalize(),
  'PANTOPRAZOLE':'PANTOPRAZOLE'.capitalize(),
  'SERTRALINE':'SERTRALINE'.capitalize(),
  'ALLEGRA':'ALLEGRA'.capitalize(),
  'PROZAC':'PROZAC'.capitalize(),
  'unk':'None',
  'None listed':'None',
  'UNK':'None',
  'nothing ':'None',
  'Not reported':'None',
  'NOT REPORTED':'None',
  'ASPIRIN [ACETYLSALICYLIC ACID]':'Aspirin',
  'BABY ASPIRIN':'Aspirin',
  'Pfizer, Inc. EUA 027034; Pfizer, Inc. EUA 027034; Pfizer, Inc. EUA 027034; Pfizer, Inc. EUA 027034':'None',
  'Pfizer, Inc. EUA 027034; Pfizer, Inc. EUA 027034; Pfizer, Inc. EUA 027034':'None',
  'Unkown':'None',
  'Thyroid medication':'Thyroid',
  'ORIGINAL ON FILE AT FACILITY':'None',
  'Pfizer, Inc. EUA 027034':'None',
  'Pfizer, Inc. EUA 027034; Pfizer, Inc. EUA 027034':'None',
  'none listed':'None',
  'ASPIRIN (E.C.)':'Aspirin',
  'none.':'None',
  'Multivitamin':'Vitamins',
  'MULTIVITAMIN [VITAMINS NOS]':'Vitamins',
  'PRENATAL VITAMINS [ASCORBIC ACID;BETACAROTENE;CALCIUM SULFATE;COLECALCIFEROL;CYANOCOBALAMIN;FERROUS':'Vitamins',
  'VITAMIN D [VITAMIN D NOS]':'Vitamins',
  'Vitamin C':'Vitamins',
  'yes':'None',
  'No medications':'None',
  'NONE KNOWN':'None',
  'NONE KNOWN':'None',
  'multivitamins':'Vitamins',
  'No.':'None',
  'Non':'None',
  'Vitamin D':'Vitamins',
  'LEVOTHYROXINE':'Levothyroxine',
  'Levothyroxin':'Levothyroxine',
  'levothyroxine':'Levothyroxine',
  'SYNTHROID':'Synthroid',
  'TYLENOL':'Tylenol',
  'Unknown.':'None',
  'Prenatal vitamins':'Vitamins',
  'LISINOPRIL':'Lisinopril',
  'VITAMINS NOS':'Vitamins',
  'IBUPROFEN':'Ibuprofen',
  'ADVIL [IBUPROFEN]':'Ibuprofen',
  'Advil':'Ibuprofen',
  'Prenatal vitamin':'Vitamins',
  'prenatal vitamins':'Vitamins',
  'vitamin D':'Vitamins',
  'multivitamin':'Vitamins',
  'Multi vitamin':'Vitamins',
  'MULTIVITAMINS [VITAMINS NOS]':'Vitamins',
  'Vitamin D3':'Vitamins',
  'ADVIL [IBUPROFEN]':'Vitamins',
  'Multivitamins':'Vitamins',
  'vitamins':'Vitamins',
  'Multi-vitamin':'Vitamins',
  'multi-vitamin':'Vitamins',
  'HUMIRA':'Humira',
  '?':'None',
  'Birth control':'Birth Control',
  'Birth control pill':'Birth Control',
  'birth control':'Birth Control',
  'Birth control pills':'Birth Control',
  'PRENATAL VITAMINS [MINERALS NOS;VITAMINS NOS]':'Vitamins',
  'METFORMIN':'Metformin',
  'INSULIN':'Insulin',
  'Prenatal Vitamins':'Vitamins',
  'Multi Vitamin':'Vitamins',
  'VITAMIN D NOS':'Vitamins',
  'Daily vitamin':'Vitamins',
  'Prenatal':'Vitamins',
  'zyrtec':'Zyrtec',
  'Daily multivitamin':'Vitamins',
  'Prenatal Vitamin':'Vitamins',
  'OMEPRAZOLE':'Omeprazole',
  'omeprazole':'Omeprazole',
  'ZOLOFT':'ZOLOFT'.capitalize(),
  'PREDNISONE':'PREDNISONE'.capitalize(),
  'AMLODIPINE':'AMLODIPINE'.capitalize(),
  'ELIQUIS':'ELIQUIS'.capitalize(),
  'WARFARIN':'WARFARIN'.capitalize(),
  'amlodipine':'amlodipine'.capitalize(),
  'metformin':'metformin'.capitalize(),
  'BENADRYL':'BENADRYL'.capitalize()
}
df_data_clean['OTHER_MEDS'] = df_data_clean['OTHER_MEDS'].replace(replace_dist_OTHER_MEDS)

#df_data['OTHER_MEDS'].value_counts()[:90].sort_values(ascending=False)



In [9]:
df_data_selected_cols= ['VAERS_ID', 'AGE_YRS' ,'DIED', 'L_THREAT' , 'HOSPITAL','HOSPDAYS','OTHER_MEDS','HISTORY']
selected_meds = ['None', 'Vitamins','Levothyroxine','Tylenol','Birth Control','Synthroid','Ibuprofen','Zyrtec','Lisinopril','Metformin','Lexapro']
hist_names = ['Unknown','Asthma','Hypertension','Diabetes','Hypothyroidism','Allergy','COVID-19','Migraines','Obesity','Anxiety','HTN']
df_data_clean_final = df_data_clean[df_data_selected_cols]
df_data_clean_final= df_data_clean_final[(df_data_clean_final['HISTORY'].isin(hist_names)) & (df_data_clean_final['OTHER_MEDS'].isin(selected_meds))]
df_data_clean_final.DIED = df_data_clean_final.DIED.fillna('N')

df_data_clean_final.L_THREAT = df_data_clean_final.L_THREAT.fillna('N')

df_data_clean_final.HOSPITAL = df_data_clean_final.HOSPITAL.fillna('N')

df_data_clean_final.HOSPDAYS = df_data_clean_final.HOSPDAYS.fillna(0)

#df_data_clean_final['HOSPITAL'].value_counts()[:200].sort_values(ascending=False)

In [ ]:
# plot_frec_col (df_data_clean_final, 'HOSPITAL')
# plot_frec_col (df_data_clean_final, 'DIED')
# plot_frec_col (df_data_clean_final, 'L_THREAT')

In [ ]:
mapping = {'N': 0, 'Y': 1}
df_data_clean_final.replace({'HOSPITAL': mapping, 'L_THREAT': mapping , 'DIED':mapping})
df_data_clean_final.head()

In [10]:

df_data_clean_final['AGE_YRS'] =pd.to_numeric(df_data_clean_final['AGE_YRS'])
df_data_clean_final['DIED'] = df_data_clean_final['DIED'].astype('category').apply(lambda x : pd.factorize(x)[0])
df_data_clean_final['L_THREAT'] = df_data_clean_final['L_THREAT'].astype('category').apply(lambda x : pd.factorize(x)[0])
df_data_clean_final['HOSPITAL'] = df_data_clean_final['HOSPITAL'].astype('category').apply(lambda x : pd.factorize(x)[0])
df_data_clean_final['HOSPDAYS'] = pd.to_numeric(df_data_clean_final['HOSPDAYS'])

df_data_clean_final.dtypes
df_data_clean_final.columns

TypeError: unhashable type: 'numpy.ndarray'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


TypeError: unhashable type: 'numpy.ndarray'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


TypeError: unhashable type: 'numpy.ndarray'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


Index(['VAERS_ID', 'AGE_YRS', 'DIED', 'L_THREAT', 'HOSPITAL', 'HOSPDAYS', 'OTHER_MEDS', 'HISTORY'], dtype='object')

In [ ]:
# df_data_clean_final['AGE_YRS'].corr(df_data_clean_final['HOSPITAL'])
#df_data_clean_final.head()
# df_data_clean_final.DIED = pd.get_dummies(df_data_clean_final['DIED'])
# df_data_clean_final.L_THREAT = pd.get_dummies(df_data_clean_final['L_THREAT'])
# df_data_clean_final.HOSPITAL = pd.get_dummies(df_data_clean_final['HOSPITAL'])
# df_data_clean_final.OTHER_MEDS = pd.get_dummies(df_data_clean_final['OTHER_MEDS'])
# df_data_clean_final.HISTORY = pd.get_dummies(df_data_clean_final['HISTORY'])

In [ ]:
df_data_clean_final[['AGE_YRS', 'DIED', 'L_THREAT', 'HOSPITAL', 'HOSPDAYS', 'OTHER_MEDS', 'HISTORY']].apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)

In [ ]:
# calculate the crrolation with the traget column *HOSPITAL*
correlations = df_data_clean_final.corrwith(df_data_clean_final.HOSPITAL)

# remove the correlations with vaule 1 *remove the correlation between HOSPITAL HOSPITAL*
correlations = correlations[correlations!=1]

# Split the correlations into positive and negative
positive_correlations = correlations[correlations >0].sort_values(ascending = False)
negative_correlations =correlations[correlations<0].sort_values(ascending = False)

# print the coorelations 
print('Most Positive Correlations:\n', positive_correlations)
print('\nMost Negative Correlations:\n', negative_correlations)